# CNN for Fluo Classification

### Install Dependencies

In [ ]:
%matplotlib inline

In [ ]:
!pip install mxnet-cu101
!pip install gluoncv

     |███████████████████████████████▌| 834.1MB 1.5MB/s eta 0:00:09tcmalloc: large alloc 1147494400 bytes == 0x662ea000 @  0x7f305212b615 0x591f47 0x4cc229 0x4cc38b 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x509918 0x50a64d
     |████████████████████████████████| 846.0MB 21kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 819kB 7.7MB/s 


In [ ]:
import mxnet as mx
import numpy as np
import os, time, shutil
import matplotlib.pyplot as plt
from PIL import Image

from mxnet import gluon, image, init, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs
from gluoncv.model_zoo import get_model

### Mount Google Drive

In [ ]:
import os
from google.colab import files, drive   
import pandas as pd

# mount the google drive to my Colab session
drive.mount('/content/gdrive')

home_path = '/content/gdrive/Shared drives/Embryo_data'

# use the google drive in my Colab session
print(os.listdir(home_path))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['mxnet_cnn2d_embryo_58_fine_tune_data_aug_ResNet50_v2_order_random.ipynb', 'Embryo1', 'Embryo3', 'Embryo12', 'Embryo13', 'Embryo16', 'Embryo20', 'Embryo19', 'Embryo18', 'Embryo24', 'Embryo40', 'Embryo39', 'Embryo42', 'Embryo52', 'Embryo50', 'Embryo49', 'Embryo45', 'Embryo46', 'Embryo47', 'Embryo23', 'Embryo33', 'Embryo25', 'Embryo95', 'Embryo97', 'Embryo96', 'Embryo98', 'Embryo101', 'Embryo99', 'Embryo100', 'Embryo102', 'Embryo76', 'Embryo78', 'Embryo81', 'Embryo79', 'Embryo80', 'Embryo82', 'Embryo84', 'Embryo83', 'Embryo85', 'Embryo87', 'Embryo88', 'Embryo92', 'Embryo94', 'Embryo93', 'raw', 'embryo_info_CS101.xlsx']


### Get Data

In [ ]:
# Fixing the random seed
mx.random.seed(42)
np.random.seed(42)

# Load info about videos
cols_to_skip = ['remarks']
video_time_info = pd.read_excel(f'{home_path}/embryo_info_CS101.xlsx', usecols=lambda x: x not in cols_to_skip)

# Fill list of embryo indices and corresponding total t val
embryo_inds = []
t_nums = []
for pd_idx in range(len(video_time_info)):
    # print(f'{pd_idx}/{len(video_time_info)}')
    if all(video_time_info[['if_full_injected', 'fluo_quality_of_z_max_sum', 'fluo_quality_of_raw_png', 'if_healthy']].to_numpy()[pd_idx]):
        # embryo_idx, t_num, c_fluo, c_bf = embryo_data[["embryo_index", "t_num", "fluo_channel", "DIC_channel"]].values[pd_idx]
        embryo_idx, t_num = video_time_info[['embryo_index', 't_num']].to_numpy()[pd_idx]
        embryo_inds.append(embryo_idx)
        t_nums.append(t_num)

# t_num = video_time_info.loc[:,'t_num'].to_numpy()
print("Embryo indices: ", embryo_inds)
print("Corresponding total t:", list(t_nums))
assert len(embryo_inds) == len(t_nums)

embryo_inds = np.array(embryo_inds)
t_nums = np.array(t_nums)

Embryo indices:  [1, 3, 12, 13, 16, 18, 19, 20, 24, 39, 40, 42, 45, 46, 47, 49, 50, 52, 76, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]
Corresponding total t: [21, 21, 143, 143, 143, 143, 143, 143, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109]


In [ ]:
## Train-Val-Test Split

# Randomly shuffle
p = np.random.permutation(len(embryo_inds))
embryo_inds_rand = np.zeros_like(embryo_inds)
t_nums_rand = np.zeros_like(t_nums)

for i in range(len(embryo_inds)):
  embryo_inds_rand[i] = embryo_inds[p[i]]
  t_nums_rand[i] = t_nums[p[i]]

t_nums_rand_cumsum = np.cumsum(t_nums_rand)
# print(t_nums_rand_cumsum)

test_split_point = t_nums_rand_cumsum[-1]*0.83
temp = abs(t_nums_rand_cumsum-test_split_point)
test_idx = np.argmin(temp)
print("test idx: ", test_idx)

val_split_point = t_nums_rand_cumsum[-1]*0.7
temp = abs(t_nums_rand_cumsum-val_split_point)
val_idx = np.argmin(temp)
print("val idx: ", val_idx)

train_embryos = embryo_inds_rand[:val_idx]
val_embryos = embryo_inds_rand[val_idx:test_idx]
test_embryos = embryo_inds_rand[test_idx:]
print(train_embryos)
print(val_embryos)
print(test_embryos)

test idx:  31
val idx:  26
[ 81   1  85  16  95  76  40  96  49  45  94  39  82  18  80  93  92  98
 100  99  50  97  12  88  20  42]
[19 47 87 78  3]
[ 79  84 102  46  83  13  52 101  24]


In [ ]:
# Parent directories of the processed *.npy files
processed_path = f'{home_path}/processed'
polar_processed_path = f'{processed_path}/polarization'

# Directories to load data from
data_path = f'{processed_path}/fluo_data/middle'
pol_path = f'{processed_path}/polarization'

# Directories to write to
train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'val')
test_path = os.path.join(data_path, 'test')


# TODO: get middle dynamically, or re-make subdirectory structure in the new drive

### Save NP Data as PNG for Training

In [ ]:
# Actually create the images

def save_nps_as_png(embryos, save_path):
    '''
    embryos: subset of embryo_inds_rand... train, val, test
    save_path: path to save png to... data_path + {'train', 'val', 'test'}
    '''
    for i in range(len(embryos)):
        embryo_idx = embryos[i]
        embryo_path = f'{data_path}/embryo_{embryo_idx}.npy'
        embryo_pol_path = f'{pol_path}/embryo_{embryo_idx}.npy'
        embryo = np.load(embryo_path)
        embryo_pol = np.squeeze(np.load(embryo_pol_path)).astype(int)
        embryo = embryo.astype(np.float64) / np.max(embryo) # normalize the data to 0 - 1
        embryo = 255 * embryo # Now scale by 255
        embryo = embryo.astype(np.uint8)
        print(embryo_idx)
        for t in range(np.shape(embryo)[2]):
            pol = embryo_pol[t]
            img = Image.fromarray(embryo[:,:,t], 'L')
            img_path = f'{save_path}/{pol}/embryo_{embryo_idx}_{t}.png'
            img.save(img_path)

save_nps_as_png(train_embryos, train_path)
save_nps_as_png(val_embryos, val_path)
save_nps_as_png(test_embryos, test_path)

FileNotFoundError: ignored

### Set Hyperparameters

In [ ]:
classes = 2

epochs = 15
lr = 0.001
per_device_batch_size = 16
momentum = 0.9
wd = 0.0001

lr_factor = 0.75
lr_steps = [10, 20, 30, np.inf]

num_gpus = 1
num_workers = 8
ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
batch_size = per_device_batch_size * max(num_gpus, 1)
print(ctx)

[gpu(0)]


Things to keep in mind:

1. ``epochs = 5`` is just for this tutorial with the tiny dataset. please change it to a larger number in your experiments, for instance 40.
2. ``per_device_batch_size`` is also set to a small number. In your experiments you can try larger number like 64.
3. remember to tune ``num_gpus`` and ``num_workers`` according to your machine.
4. A pre-trained model is already in a pretty good status. So we can start with a small ``lr``.

### Data Augmentation

In transfer learning, data augmentation can also help.
We use the following augmentation in training:

2. Randomly crop the image and resize it to 224x224
3. Randomly flip the image horizontally
4. Randomly jitter color and add noise
5. Transpose the data from height*width*num_channels to num_channels*height*width, and map values from [0, 255] to [0, 1]
6. Normalize with the mean and standard deviation from the ImageNet dataset.




In [ ]:
transform_train = transforms.Compose([
    transforms.Resize(600, keep_ratio=True),
    transforms.CenterCrop(512),

    transforms.RandomFlipLeftRight(), # Randomly flip the image horizontally
    transforms.RandomFlipTopBottom(),
    transforms.RandomLighting(0.1), # Add AlexNet-style PCA-based noise to an image
    transforms.RandomContrast(0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(600, keep_ratio=True),
    transforms.CenterCrop(512),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

### Data Loaders

With the data augmentation functions, we can define our data loaders:



In [ ]:
train_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(train_path).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, num_workers=num_workers)

val_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(val_path).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, num_workers = num_workers)

test_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(test_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

print(len(train_data) + len(val_data) + len(test_data))

63


Note that only ``train_data`` uses ``transform_train``, while
``val_data`` and ``test_data`` use ``transform_test`` to produce deterministic
results for evaluation.

## Training

### Define Model

Simple CNN for now

In [ ]:
import mxnet.ndarray as F

class Net(gluon.Block):
    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            # layers created in name_scope will inherit name space
            # from parent layer.
            self.conv1 = nn.Conv2D(20, kernel_size=(5,5))
            self.pool1 = nn.MaxPool2D(pool_size=(2,2), strides = (2,2))
            self.conv2 = nn.Conv2D(50, kernel_size=(5,5))
            self.pool2 = nn.MaxPool2D(pool_size=(2,2), strides = (2,2))
            self.fc1 = nn.Dense(500)
            self.fc2 = nn.Dense(10)

    def forward(self, x):
        x = self.pool1(F.tanh(self.conv1(x)))
        x = self.pool2(F.tanh(self.conv2(x)))
        # 0 means copy over size from corresponding dimension.
        # -1 means infer size from the rest of dimensions.
        x = x.reshape((0, -1))
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        return x

### Initialize Model, Parameters, Trainer

In [ ]:
net = Net()

optimizer = 'sgd' # 'nag': Nesterov accelerated gradient descent
optimizer_params = {'learning_rate': lr, 'wd': wd, 'momentum': momentum} # Set parameters

trainer = gluon.Trainer(net.collect_params(), optimizer, optimizer_params)

### Define Evaluation Function for Test/Val Accuracy

In [ ]:
def test(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        metric.update(label, outputs)

    return metric.get()

### Training Loop

Following is the main training loop. It is the same as the loop in
`CIFAR10 <dive_deep_cifar10.html>`__
and ImageNet.

In [ ]:
lr_counter = 0
num_batch = len(train_data)
train_acc_lst = []
train_loss_lst = []
val_acc_lst = []

for epoch in range(epochs):
    if epoch == lr_steps[lr_counter]:
        trainer.set_learning_rate(trainer.learning_rate*lr_factor)
        lr_counter += 1

    tic = time.time()
    train_loss = 0
    metric.reset()

    for i, batch in enumerate(train_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        with ag.record():
            outputs = [net(X) for X in data]
            loss = [L(yhat, y) for yhat, y in zip(outputs, label)]
        for l in loss:
            l.backward()

        trainer.step(batch_size)
        train_loss += sum([l.mean().asscalar() for l in loss]) / len(loss)

        metric.update(label, outputs)

    _, train_acc = metric.get()
    train_loss /= num_batch

    _, val_acc = test(net, val_data, ctx)

    train_acc_lst.append(train_acc)
    train_loss_lst.append(train_loss)
    val_acc_lst.append(val_acc)
    print('[Epoch %d] Train-acc: %.3f, loss: %.3f | Val-acc: %.3f | time: %.1f' %
             (epoch, train_acc, train_loss, val_acc, time.time() - tic))

_, test_acc = test(net, test_data, ctx)
print('[Finished] Test-acc: %.3f' % (test_acc))

[Epoch 0] Train-acc: 0.992, loss: 0.028 | Val-acc: 0.941 | time: 37.2


### Plot Results

In [ ]:
plt.figure()
plt.plot(np.arange(0,epochs,1),test_acc_lst,'r')
plt.ylabel('Training Accuracy')
plt.xlabel('Epoch')
plt.title('CNN Classify Fluo Polarization')
plt.show()

In [ ]:
plt.figure()
plt.plot(np.arange(0,epochs,1),test_loss_lst,'r')
plt.ylabel('Training Loss')
plt.xlabel('Epoch')
plt.title('CNN Classify Fluo Polarization')
plt.show()

In [ ]:
plt.figure()
plt.plot(np.arange(0,epochs,1),val_acc_lst,'r')
plt.ylabel('Validation Accuracy')
plt.xlabel('Epoch')
plt.title('CNN Classify Fluo Polarization')
plt.show()